## @dataclass (clean configs and state)
- Why: avoids hand-written __init__, keeps configs tidy & typed.

In [2]:
from dataclasses import dataclass
from typing import List, Optional

# before
class SerialConfig_before:
    def __init__(self, port: Optional[str] = None, baud: int = 115200, joints: List[str] = None):
        self.port = port
        self.baud = baud
        self.joints = joints

# after
@dataclass
class SerialConfig_after:
    port: Optional[str] = None
    baud: int = 115200
    joints: List[str] = None


In [3]:
# Example usage
config1 = SerialConfig_before(port="/dev/ttyUSB0", baud=9600, joints=["joint1", "joint2"])
config2 = SerialConfig_after(port="/dev/ttyUSB0", baud=9600, joints=["joint1", "joint2"])
print(config1.port, config1.baud, config1.joints)
print(config2.port, config2.baud, config2.joints)

/dev/ttyUSB0 9600 ['joint1', 'joint2']
/dev/ttyUSB0 9600 ['joint1', 'joint2']


## Validate & normalize in __post_init__

In [4]:
from dataclasses import dataclass

@dataclass
class Gains:
    kp: float
    ki: float = 0.0
    kd: float = 0.0

    def __post_init__(self):
        if self.kp < 0 or self.ki < 0 or self.kd < 0:
            raise ValueError("Gains must be non-negative")


In [5]:
# example usage
gains = Gains(kp=1.0, ki=0.1, kd=0.01)
print(gains)

Gains(kp=1.0, ki=0.1, kd=0.01)


In [6]:
gains_invalid = Gains(kp=-1.0)

ValueError: Gains must be non-negative

## Immutability (frozen=True) and hashing

- Hashing is the process of converting an object into a fixed-size integer (the hash) using a hash function
- A good hash function is deterministic (same input -> same hash each run), fast, and spreads values to avoid collisions. 
- Hashes are used for fast lookup in hash tables (dict, set). 
- if two objects compare equal, they must have the same hash; if $a==b⇒hash(a)=hash(b)$.

In [7]:
from dataclasses import dataclass, field
from datetime import datetime

@dataclass(frozen=True, slots=True)
class Book:
    # Identity field(s)
    isbn: str
    # Non-identity metadata (ignored for == and hash)
    title: str = field(compare=False)
    price: float = field(compare=False)
    added_at: datetime = field(default_factory=datetime.utcnow,
                               compare=False, repr=False)

    def __post_init__(self):
        # Normalize so logically-equal objects compare equal & hash equal
        norm = self.isbn.replace("-", "").strip().upper()
        object.__setattr__(self, "isbn", norm)

    def __hash__(self) -> int:
        # Equal books (same normalized ISBN) must have the same hash
        return hash(self.isbn)


In [8]:
# example usage
b1 = Book("978-0-13-235088-4", "Clean Code", 450.0)
b2 = Book("9780132350884", "Clean Code (2nd)", 499.0)

print(b1 == b2)                  # True (same normalized ISBN)
print(hash(b1) == hash(b2))      # True
s = {b1, b2}
print(len(s))                    # 1 (deduplicated in a set)

d = {b1: "available"}
print(d[b2])                     # "available" (same key after normalization)


True
True
1
available


## Ordering and selective comparisons

In [10]:
from dataclasses import dataclass, field

@dataclass(order=True)
class Job:
    priority: int
    name: str = field(compare=False)

In [11]:
j1 = Job(priority=1, name="Low priority task")
j2 = Job(priority=10, name="High priority task")
print(j1 < j2)  # True (compares by priority only)

True


## Inheritance allows you to reuse and extend configuration classes (configs) and mixins.  

- **Configs**: Base config classes define shared fields and validation logic. You can inherit and specialize them for different use cases, keeping code DRY and maintainable.
- **Mixins**: Mixins are reusable classes that add specific behavior (methods/properties) to multiple classes via inheritance, without being the main parent. They help compose functionality in a modular way.Inheritance (configs & mixins)

### Configs (base classes)

In [13]:
from dataclasses import dataclass

@dataclass
class BaseConfig:
    host: str = "localhost"
    port: int = 8000
    timeout: float = 5.0  # seconds

@dataclass
class DevConfig(BaseConfig):
    debug: bool = True
    host: str = "localhost"
    port: int = 8000

@dataclass
class ProdConfig(BaseConfig):
    debug: bool = False
    host: str = "0.0.0.0"
    port: int = 80


### Mixins (add-on behaviors)

In [14]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str = "localhost"
    age: int = 0
    email: str = "user@example.com"

@dataclass
class EmployeeMixin:
    employee_id: str = "E000"
    department: str = "General"

@dataclass
class Employee(Person, EmployeeMixin):
    position: str = "Staff"

In [18]:
# example (Indian context)
emp = Employee(name="Ram", age=30, email="ram@example.com", department="IT", employee_id="IN001", position="Developer")
print(emp)

Employee(employee_id='IN001', department='IT', name='Ram', age=30, email='ram@example.com', position='Developer')
